# BC(Behavior Cloning)を使用した模倣学習
- エキスパートの動作を模倣して学習するための一連の処理
- 使用環境：CartPole（ポールが倒れないようにカートを動かす制御タスク）
- 使用ライブラリ：imitation, stable-baselines3, gymnasium

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
from stable_baselines3.common.evaluation import evaluate_policy  # ポリシーの評価を行うための関数

from imitation.policies.serialize import load_policy  # 事前に学習されたポリシーをロードするための関数
from imitation.util.util import make_vec_env  # ベクトル化された環境を作成するための関数
from imitation.data.wrappers import RolloutInfoWrapper  # ロールアウト中に追加情報を保持できるラッパー

/home/lilin/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# step1. 環境作成
- CartPole(カートとポール)タスクの環境を作成
- "seals"は、SEALSベンチマーク環境

In [ ]:
# 環境の設定
env = make_vec_env(
    "seals:seals/CartPole-v0",      # 使用する環境名
    rng=np.random.default_rng(),    # Numpyの乱数生成器を指定
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)  # エピソードごとの情報（リターンなど）を取得できるようにラップ
    ],  # needed for computing rollouts later
)

## step2. 事前学習済みのエキスパートポリシーをロード
- Hugging Face上で公開されている、優れたPROアルゴリズムで学習されたポリシーをロード
    - https://huggingface.co/HumanCompatibleAI

- エキスパートが優秀か確認する
- 通常、達成可能な最大値である500の報酬を取得できる

In [3]:
# ポリシーのロード
expert = load_policy(
    "ppo-huggingface",                  # 使用するポリシーの名前
    organization="HumanCompatibleAI",   # 公開している組織名
    env_name="seals/CartPole-v0",       # 環境名
    venv=env,                           # 作成済みの環境を指定（ベクトル化済み）
)

# エキスパートポリシーの性能を確認（行動模倣する価値があるか？）
# CartPole環境では最大報酬が500。これに近いスコアを出せていれば、十分に優秀。
reward, _ = evaluate_policy(expert, env, 10)  # 10エピソード評価
print(f"エキスパートの平均報酬: {reward:.2f}")

/home/lilin/anaconda3/envs/stable_baselines/lib/python3.10/site-packages/stable_baselines3/common/on_policy_algorithm.py:150: UserWarning: You are trying to run PPO on the GPU, but it is primarily intended to run on the CPU when not using a CNN policy (you are using ActorCriticPolicy which should be a MlpPolicy). See https://github.com/DLR-RM/stable-baselines3/issues/1245 for more info. You can pass `device='cpu'` or `export CUDA_VISIBLE_DEVICES=` to force using the CPU.Note: The model will train, but the GPU utilization will be poor and the training might take longer than on CPU.
  warnings.warn(


エキスパートの平均報酬: 500.00


## step3. エキスパートの行動データ（軌跡）を収集
- 以上で、エキスパートを使って行動データをサンプリングできるようになった
- 動作のクローン化に必要なのは個々の遷移だけなので、**平坦化する**
    - `imitation`には、繊維の収集を非常に簡単にするエスパー関数が多数用意されている
    - 最初に50エピソードのロールアウトを収集
    - 次にトレーニングに必要な遷移だけ平坦化
- `rollout`関数はベクトル化された環境を必要とする
    - 注意：各環境の周囲に`RolloutInfoWrapper`が必要


In [4]:
from imitation.data import rollout

rng = np.random.default_rng()  # 別の乱数生成器

# エキスパートに50エピソード分のプレイを行わせて、行動データを収集
rollouts = rollout.rollout(
    expert, # エキスパートポリシー
    env,    # 環境
    rollout.make_sample_until(min_timesteps=None, min_episodes=50),  # 50エピソード分のデータを収集するための条件
    rng=rng,
)

# 軌跡を「遷移」（Transition）として扱いやすいようにフラットな構造に変換
transitions = rollout.flatten_trajectories(rollouts)

# 軌跡データの構造を確認
print(
    f"""
    ロールアウト関数は{len(rollouts)}個の{type(rollouts[0])}のリストを生成しました。
    このリストはフラット化後、{len(transitions)}個のトランザクションを含む{type(transitions)} オブジェクトに変換されました。
    トランザクションオブジェクトには、次の配列が含まれています: {', '.join(transitions.__dict__.keys())}.
    """
)


    ロールアウト関数は56個の<class 'imitation.data.types.TrajectoryWithRew'>のリストを生成しました。
    このリストはフラット化後、28000個のトランザクションを含む<class 'imitation.data.types.Transitions'> オブジェクトに変換されました。
    トランザクションオブジェクトには、次の配列が含まれています: obs, acts, infos, next_obs, dones.
    


## step4. BCトレーナーの構築
- 遷移を収集したら、動作の複製アルゴリズムを設定する

In [5]:
from imitation.algorithms import bc

# 行動模倣（Behavior Cloning）アルゴリズムのインスタンスを作成
# トレーニングデータとして、エキスパートのデモンストレーションデータを使用
bc_trainer = bc.BC(
    observation_space=env.observation_space,    # 環境の観測空間
    action_space=env.action_space,              # 環境の行動空間（離散/連続）
    demonstrations=transitions,                 # エキスパートのデモンストレーションデータ（遷移）
    rng=rng,                                    # 乱数生成器 
    device="cpu",                               # 追加: すべてCPUで計算
)

- トレーニング前の模倣ポリシーの性能を確認する
    - 当然、エキスパートには及ばなさい

In [6]:
reward_before_training, _ = evaluate_policy(bc_trainer.policy, env, 10)
print(f"トレーニング前の報酬: {reward_before_training}")

トレーニング前の報酬: 89.7


- トレーニング後は、エキスパートの報酬（500）に匹敵する

In [7]:
# 実際に模倣学習を行う
# 1エポックのトレーニングを実行
# エポック数を増やすとよりよく学習されるが、時間がかかる
bc_trainer.train(n_epochs=1)  # 1エポックのトレーニング

# トレーニング後のポリシーを評価
reward_after_training, _ = evaluate_policy(bc_trainer.policy, env, 10)  
print(f"トレーニング後の報酬: {reward_after_training}")

0batch [00:00, ?batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 0         |
|    ent_loss       | -0.000693 |
|    entropy        | 0.693     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 72.5      |
|    loss           | 0.692     |
|    neglogp        | 0.692     |
|    prob_true_act  | 0.5       |
|    samples_so_far | 32        |
---------------------------------


467batch [00:01, 361.02batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 500       |
|    ent_loss       | -0.000241 |
|    entropy        | 0.241     |
|    epoch          | 0         |
|    l2_loss        | 0         |
|    l2_norm        | 95.3      |
|    loss           | 0.126     |
|    neglogp        | 0.126     |
|    prob_true_act  | 0.897     |
|    samples_so_far | 16032     |
---------------------------------


875batch [00:02, 379.89batch/s]


トレーニング後の報酬: 500.0


# トレーニング後のエージェントの動作を動画で可視化

In [8]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecVideoRecorder
import os

# 動画保存先ディレクトリ
video_dir = "./videos/"
os.makedirs(video_dir, exist_ok=True)

# render_mode を "rgb_array" に設定して環境を作成
video_env = DummyVecEnv([lambda: gym.make("CartPole-v1", render_mode="rgb_array")])

# VecVideoRecorder でラップ
video_env = VecVideoRecorder(
    video_env,
    video_folder=video_dir,
    record_video_trigger=lambda step: step == 0,
    video_length=500,
    name_prefix="bc_agent",
)

# ビヘイビアクローンされたエージェントのポリシーを使って実行
obs = video_env.reset()
done = False
while not done:
    action, _ = bc_trainer.policy.predict(obs)
    obs, _, dones, _ = video_env.step(action)
    done = dones[0]

# 動画を保存
video_env.close()
print(f"動画を {video_dir} に保存しました。")


Saving video to /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/Behavior_Cloning/videos/bc_agent-step-0-to-step-500.mp4
MoviePy - Building video /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/Behavior_Cloning/videos/bc_agent-step-0-to-step-500.mp4.
MoviePy - Writing video /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/Behavior_Cloning/videos/bc_agent-step-0-to-step-500.mp4



MoviePy - Done !
MoviePy - video ready /home/lilin/satoya_ws/study_deep_learn/StudyStableBaselines/workspace/Behavior_Cloning/videos/bc_agent-step-0-to-step-500.mp4
動画を ./videos/ に保存しました。


In [9]:
from IPython.display import Video

# 動画ファイルのパス（例: bc_agent-step-0-to-step-500.mp4）
video_path = os.path.join(video_dir, "bc_agent-step-0-to-step-500.mp4")
Video(video_path, embed=True)

## 備考
- Behavior Cloningは「観測 -> 行動」を**教師あり学習**で学習する手法
- エキスパートの行動履歴（観測と対応する行動）を教師データとして使用
- 強化学習のように、「報酬」から学ぶのではなく「正解の行動」を直接学ぶ
    - エキスパートの質や量が非常に重要
- SEALSの環境は、観測の安定性が保証されており、模倣学習に最適

# 参考
https://imitation.readthedocs.io/en/latest/tutorials/1_train_bc.html